# Capstone Project - Gentrification of Berlin seen through the lens of vegan/vegetarian  resturants.

## What the volume and popularity of vegan resturants tells about the demographics of Berlin's boroughs.

# Table of Contents 
* [Introduction](#introduction)
* [Data](#data)
* [Clustering](#Cluster)
* [Conclusion](#conclusion)

## Introduction<a name="introduction"></a>

### Background
Berlin is a city that has seen vast changes since the fall of the berlin wall and the end of the cold war. On such changes is a percived genrtrification of part of the city which have gained a reputation for being a more bohemian, young and "hippster". Another widely held perception is that within this demopgraphic of young and "hippster" type people is that veganism and alternative diets are popular. This project aims to test these assumptions and gather the most vegan friendly areas of Berlin using available data. 

### Problem
The problem may sound trivial, however it is hugley import to understand the demographics of cities in order to understand it's people, needs and problems. This is escpcially true in Berlin that has seen vast changes since the reunification of Germany in the early 1990s which created much change in berlin including a influx of new arrivals or the re-settlement of people into the urban centers which inturn has lead to the displacement of others due to inflating rent prices. Such phenomena has been phrased as gentrification (1) and has put ressures on local government who try to balance urban rewnewal whilst limiting the negative effects (2). 

1) Zerofsky (2019) - https://www.newyorker.com/news/dispatch/the-causes-and-consequences-of-berlins-rapid-gentrification

2) Levine (2004) - http://rohcavamaintenant.free.fr/USB%20KEY%20Fahriye/GENTRIFICATION/Government%20Policy-The%20Local%20State-and%20Gentrification.pdf

### Interest
The results of the research could provide helpful for government, town planners, businesses and residents as a guide to more tradtional areas with more gentrified and younger areas of the city.  

## Data acquisition and cleaning<a name="data"></a>

### Data Source and Reasoning

Based on definition of our problem, factors that will influence our decission are:

- number of existing restaurants in the neighborhood (any type of restaurant)
- number of specialty vegan restaurants in the neighborhood, if any
- distance of neighborhood from city center
- Demographics of neighborhood

To work this I will primarly be sourcing location based data through the Foursquare API as well as availble wiki data. There are a few important data points we can use to attempt to answer the question above. 

1) As mentioned above  would like to establish frequancy of types of restaurants (vegan and other) in the Berlin boroughs. This will alow calcualtion of restaurants per capita. Such data will be gathered through geospatial data (longitudinal and latitudinal coordinates) which can be used in the  in foursquare database which can map places of note on maps including venues such as restaurants and cafes, museums and shops.

2) Another key point of interest for this research in the foursquare API data is restaurant reviews. By comparing the reviews and and ratings of both vegan and other restaurants in the city we may be able to establish the degree of populararity of restaurants potentially giving us a idea of tastes the residents of Berlin.

3) Demographic data including population and ethnicity of the boroughs will allow us to compare and contrast restaurant data. This data will be sourced by webscrapping wikipedia. It is also this table that will give us a defintive list of the berlin bourghs which can be used to search the centre coordinates of each boroughs.  

### Data Collection and Cleaning 
The main Python modules that will be used for data collection, cleaning and analysis include:

- Request
- BeautifulSoup
- Pandas
- Numpy
- Geocoder
- Folium
- the Foursquare API

They will be used for collecting and cleaning the data, reorganizing and transforming data including One-Hot-Encoding.

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup
import xml
print('Libraries imported.')

Libraries imported.


In [2]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import geopy
print('Geocoder Installed')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-1_llvm
  alsa-lib           conda-forge/linux-64::alsa-lib-1.2.3-h516909a_0
  appdirs            conda-forge/noarch::appdirs-1.4.4-pyh9f0ad1d_0
  argon2-cffi        conda-forge/linux-64::argon2-cffi-20.1.0-py37h5e8e339_2
  async_generator    conda-forge/noarch::async_generator-1.10-py_0
  backports          conda-forge/noarch::backports-1.0-py_2
  backports.functoo~ conda-forge/noarch::backports.functools_lru_cache-1.6.4-pyhd8ed1ab_0
  brotli             conda-forge/

Preparing transaction: done
Verifying transaction: done
Executing transaction: | b'Exception while loading config file /var/pod/.ws/ax-ext/config/wscloud/jupyter_notebook_config.py\n    Traceback (most recent call last):\n      File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/traitlets/config/application.py", line 737, in _load_config_files\n        config = loader.load_config()\n      File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/traitlets/config/loader.py", line 616, in load_config\n        self._read_file_as_dict()\n      File "/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/traitlets/config/loader.py", line 648, in _read_file_as_dict\n        exec(compile(f.read(), conf_filename, \'exec\'), namespace, namespace)\n      File "/var/pod/.ws/ax-ext/config/wscloud/jupyter_notebook_config.py", line 17, in <module>\n        from cdsax_jupyter_extensions.ax_log import ax_log_request\n    ModuleNotFoundError: No module named \'cdsax_jupyter_e

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from pandas.core.frame import DataFrame
print('Folium Installed')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  fol

#### Borough data - Web Scraping

In [4]:
#Scraping Berlin Boroughs and neighborhoods table from Wikipedia
link = 'https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin'
page = requests.get(link).text
soup = BeautifulSoup(page, 'lxml')
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Boroughs and neighborhoods of Berlin - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"752b7a8d-15f5-42a4-a6a9-0c6dc654ae43","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Boroughs_and_neighborhoods_of_Berlin","wgTitle":"Boroughs and neighborhoods of Berlin","wgCurRevisionId":1017945329,"wgRevisionId":1017945329,"wgArticleId":4014590,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with German-language sources (de)","Webarchive template way

In [5]:
neighborhoodList = []
for i in range(1, 13):
    table = soup.find_all('table', {'class':'sortable wikitable'})[i]
    links=table.find_all('a')[1:]
    for link in links:
        neighborhoodList.append(link.get('title'))
print(type(table))
print(type(links))
#neighborhoodList
print(neighborhoodList)

<class 'bs4.element.Tag'>
<class 'list'>
['Mitte (locality)', 'District map of Mitte', None, 'Moabit', None, 'Hansaviertel', None, 'Tiergarten (Berlin)', None, 'Wedding (Berlin)', None, 'Gesundbrunnen (Berlin)', 'Friedrichshain', 'District map of Friedrichshain-Kreuzberg', None, 'Kreuzberg', 'Prenzlauer Berg', 'District map of Pankow', None, 'Weißensee (Berlin)', None, 'Blankenburg (Berlin)', None, 'Heinersdorf', None, 'Karow (Berlin)', None, 'Stadtrandsiedlung Malchow', None, 'Pankow (locality)', None, 'Blankenfelde', None, 'Buch (Berlin)', None, 'Französisch Buchholz', None, 'Niederschönhausen', None, 'Rosenthal (Berlin)', None, 'Wilhelmsruh', 'Charlottenburg', 'District map of Charlottenbrg-Wilmersdorf', None, 'Wilmersdorf', None, 'Schmargendorf', None, 'Grunewald', None, 'Westend (Berlin)', None, 'Charlottenburg-Nord', None, 'Halensee', 'Spandau (locality)', 'District map of Spandau', None, 'Haselhorst', None, 'Siemensstadt', None, 'Staaken', None, 'Gatow', None, 'Kladow', None, 'H

In [6]:
df=pd.DataFrame()
df['Neighborhood']=neighborhoodList

df

AttributeError: 'NoneType' object has no attribute 'items'

                                  Neighborhood
0                             Mitte (locality)
1                        District map of Mitte
2                                         None
3                                       Moabit
4                                         None
5                                 Hansaviertel
6                                         None
7                          Tiergarten (Berlin)
8                                         None
9                             Wedding (Berlin)
10                                        None
11                      Gesundbrunnen (Berlin)
12                              Friedrichshain
13    District map of Friedrichshain-Kreuzberg
14                                        None
15                                   Kreuzberg
16                             Prenzlauer Berg
17                      District map of Pankow
18                                        None
19                          Weißensee (Berlin)
20           

In [7]:
df=df.dropna()
df=df.reset_index(drop=True)
df

AttributeError: 'NoneType' object has no attribute 'items'

                                  Neighborhood
0                             Mitte (locality)
1                        District map of Mitte
2                                       Moabit
3                                 Hansaviertel
4                          Tiergarten (Berlin)
5                             Wedding (Berlin)
6                       Gesundbrunnen (Berlin)
7                               Friedrichshain
8     District map of Friedrichshain-Kreuzberg
9                                    Kreuzberg
10                             Prenzlauer Berg
11                      District map of Pankow
12                          Weißensee (Berlin)
13                        Blankenburg (Berlin)
14                                 Heinersdorf
15                              Karow (Berlin)
16                   Stadtrandsiedlung Malchow
17                           Pankow (locality)
18                                Blankenfelde
19                               Buch (Berlin)
20           

In [8]:
# delete the rows that contain "District map of"
df_str=df.astype(str)
df_neg= df_str[df_str['Neighborhood'].str.contains('District map of')]
test1= list(df_neg.Neighborhood)
test2=list(df_str.Neighborhood)
ret=list(set(test2)^set(test1))
df = df_str[df_str.Neighborhood.isin(ret)]
df

AttributeError: 'NoneType' object has no attribute 'items'

                  Neighborhood
0             Mitte (locality)
2                       Moabit
3                 Hansaviertel
4          Tiergarten (Berlin)
5             Wedding (Berlin)
6       Gesundbrunnen (Berlin)
7               Friedrichshain
9                    Kreuzberg
10             Prenzlauer Berg
12          Weißensee (Berlin)
13        Blankenburg (Berlin)
14                 Heinersdorf
15              Karow (Berlin)
16   Stadtrandsiedlung Malchow
17           Pankow (locality)
18                Blankenfelde
19               Buch (Berlin)
20        Französisch Buchholz
21           Niederschönhausen
22          Rosenthal (Berlin)
23                 Wilhelmsruh
24              Charlottenburg
26                 Wilmersdorf
27               Schmargendorf
28                   Grunewald
29            Westend (Berlin)
30         Charlottenburg-Nord
31                    Halensee
32          Spandau (locality)
34                  Haselhorst
35                Siemensstadt
36      

In [9]:
df=df.reset_index(drop=True)
df

AttributeError: 'NoneType' object has no attribute 'items'

                 Neighborhood
0            Mitte (locality)
1                      Moabit
2                Hansaviertel
3         Tiergarten (Berlin)
4            Wedding (Berlin)
5      Gesundbrunnen (Berlin)
6              Friedrichshain
7                   Kreuzberg
8             Prenzlauer Berg
9          Weißensee (Berlin)
10       Blankenburg (Berlin)
11                Heinersdorf
12             Karow (Berlin)
13  Stadtrandsiedlung Malchow
14          Pankow (locality)
15               Blankenfelde
16              Buch (Berlin)
17       Französisch Buchholz
18          Niederschönhausen
19         Rosenthal (Berlin)
20                Wilhelmsruh
21             Charlottenburg
22                Wilmersdorf
23              Schmargendorf
24                  Grunewald
25           Westend (Berlin)
26        Charlottenburg-Nord
27                   Halensee
28         Spandau (locality)
29                 Haselhorst
30               Siemensstadt
31                    Staaken
32        

In [10]:
print('The dataframe has {} neighborhoods.'.format(
        
    df.shape[0])
)

The dataframe has 96 neighborhoods.


In [12]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhoodlist) for neighborhoodlist in df["Neighborhood"].tolist() ]
coords

[[52.52119000000005, 13.424140000000023],
 [52.52570000000003, 13.340050000000076],
 [52.51679000000007, 13.338350000000048],
 [52.509930000000054, 13.363930000000039],
 [52.54781000000003, 13.354730000000075],
 [52.55619000000007, 13.377100000000041],
 [52.51402000000007, 13.454030000000046],
 [52.49382000000003, 13.383580000000052],
 [52.54083000000003, 13.42575000000005],
 [52.56203000000005, 13.468390000000056],
 [52.59332000000006, 13.458050000000071],
 [52.56770000000006, 13.439800000000048],
 [52.61418000000003, 13.469760000000065],
 [52.57038000000006, 13.463420000000042],
 [52.56925000000007, 13.402480000000025],
 [52.61855000000003, 13.390170000000069],
 [52.634240000000034, 13.49753000000004],
 [52.608220000000074, 13.433680000000038],
 [52.58265000000006, 13.403620000000046],
 [52.599770000000035, 13.377760000000023],
 [52.58887000000004, 13.369490000000042],
 [52.51915000000008, 13.306390000000022],
 [52.48976000000005, 13.31519000000003],
 [52.47732000000008, 13.287980000

In [13]:
print(type(coords))

<class 'list'>


In [14]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df

(96, 3)


AttributeError: 'NoneType' object has no attribute 'items'

                 Neighborhood  Latitude  Longitude
0            Mitte (locality)  52.52119   13.42414
1                      Moabit  52.52570   13.34005
2                Hansaviertel  52.51679   13.33835
3         Tiergarten (Berlin)  52.50993   13.36393
4            Wedding (Berlin)  52.54781   13.35473
5      Gesundbrunnen (Berlin)  52.55619   13.37710
6              Friedrichshain  52.51402   13.45403
7                   Kreuzberg  52.49382   13.38358
8             Prenzlauer Berg  52.54083   13.42575
9          Weißensee (Berlin)  52.56203   13.46839
10       Blankenburg (Berlin)  52.59332   13.45805
11                Heinersdorf  52.56770   13.43980
12             Karow (Berlin)  52.61418   13.46976
13  Stadtrandsiedlung Malchow  52.57038   13.46342
14          Pankow (locality)  52.56925   13.40248
15               Blankenfelde  52.61855   13.39017
16              Buch (Berlin)  52.63424   13.49753
17       Französisch Buchholz  52.60822   13.43368
18          Niederschönhausen  

In [15]:
# get the coordinates of Berlin
address = 'Berlin, Germany'
geolocator = Nominatim(user_agent="user_agent_lyan.fu.fly@gmail.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin, Germany {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin, Germany 52.5170365, 13.3888599.


In [16]:
# create map of New York using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

In [17]:
map_berlin.save('map_berlin.html')

#### Foursquare API

In [18]:
# define Foursquare Credentials and Version
CLIENT_ID = 'R3DTV1CIMP5WXVCVEYW12SWENBB144PLS5VULCQME4PPVDY3' # your Foursquare ID
CLIENT_SECRET = 'LVT0MK2IHWET15AWDDPHL1BIUUH5KZQ0V1DGBXG4KHXSV1JD' # your Foursquare Secret
VERSION = '20210706' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R3DTV1CIMP5WXVCVEYW12SWENBB144PLS5VULCQME4PPVDY3
CLIENT_SECRET:LVT0MK2IHWET15AWDDPHL1BIUUH5KZQ0V1DGBXG4KHXSV1JD


In [19]:
radius = 2000
LIMIT = 200

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.shape
venues_df.head(15)

(5645, 7)


AttributeError: 'NoneType' object has no attribute 'items'

        Neighborhood  Latitude  Longitude                     VenueName  \
0   Mitte (locality)  52.52119   13.42414            Kino International   
1   Mitte (locality)  52.52119   13.42414             Ristorante a Mano   
2   Mitte (locality)  52.52119   13.42414   Freiluftkino Friedrichshain   
3   Mitte (locality)  52.52119   13.42414                Märchenbrunnen   
4   Mitte (locality)  52.52119   13.42414                  Flying Tiger   
5   Mitte (locality)  52.52119   13.42414                Cowshed Active   
6   Mitte (locality)  52.52119   13.42414           Fat Tire Bike Tours   
7   Mitte (locality)  52.52119   13.42414      Volkspark Friedrichshain   
8   Mitte (locality)  52.52119   13.42414  Y4All - Yoga For All Mankind   
9   Mitte (locality)  52.52119   13.42414                          LUSH   
10  Mitte (locality)  52.52119   13.42414                        Luiban   
11  Mitte (locality)  52.52119   13.42414       Mama Van - Sai Gon Deli   
12  Mitte (locality)  52.

In [21]:
venues_df_restaurant=venues_df[venues_df['VenueCategory'].str.contains('Restaurant')]
venues_df_restaurant.shape
venues_df_restaurant

AttributeError: 'NoneType' object has no attribute 'items'

                   Neighborhood  Latitude  Longitude  \
1              Mitte (locality)  52.52119   13.42414   
11             Mitte (locality)  52.52119   13.42414   
14             Mitte (locality)  52.52119   13.42414   
17             Mitte (locality)  52.52119   13.42414   
21             Mitte (locality)  52.52119   13.42414   
29             Mitte (locality)  52.52119   13.42414   
30             Mitte (locality)  52.52119   13.42414   
33             Mitte (locality)  52.52119   13.42414   
37             Mitte (locality)  52.52119   13.42414   
40             Mitte (locality)  52.52119   13.42414   
42             Mitte (locality)  52.52119   13.42414   
44             Mitte (locality)  52.52119   13.42414   
52             Mitte (locality)  52.52119   13.42414   
60             Mitte (locality)  52.52119   13.42414   
62             Mitte (locality)  52.52119   13.42414   
70             Mitte (locality)  52.52119   13.42414   
73             Mitte (locality)  52.52119   13.4

In [22]:
print('There are {} uniques categories.'.format(len(venues_df_restaurant['VenueCategory'].unique())))

There are 58 uniques categories.


In [25]:
#How many restaurants are there for each type of restaurant-catogety
print(venues_df_restaurant.loc[:,'VenueCategory'].value_counts()[0:20])

Italian Restaurant               249
German Restaurant                152
Restaurant                        76
Vietnamese Restaurant             62
Greek Restaurant                  61
Falafel Restaurant                51
Fast Food Restaurant              51
Asian Restaurant                  44
Indian Restaurant                 43
Chinese Restaurant                41
Thai Restaurant                   37
Doner Restaurant                  33
Sushi Restaurant                  32
Vegetarian / Vegan Restaurant     29
Korean Restaurant                 27
Turkish Restaurant                26
Middle Eastern Restaurant         24
Mexican Restaurant                23
Seafood Restaurant                23
Japanese Restaurant               21
Name: VenueCategory, dtype: int64


In [26]:
#turn array to dataframe
venues_df_restaurant_neighborhood = pd.DataFrame(venues_df_restaurant.groupby(["Neighborhood"]).count())
venues_df_restaurant_neighborhood.sort_values(by="Latitude" , ascending=False)

AttributeError: 'NoneType' object has no attribute 'items'

                           Latitude  Longitude  VenueName  VenueLatitude  \
Neighborhood                                                               
Charlottenburg                   44         44         44             44   
Friedenau                        41         41         41             41   
Halensee                         40         40         40             40   
Grunewald                        38         38         38             38   
Wilmersdorf                      35         35         35             35   
Kreuzberg                        32         32         32             32   
Westend (Berlin)                 32         32         32             32   
Wedding (Berlin)                 32         32         32             32   
Steglitz                         31         31         31             31   
Schöneberg                       30         30         30             30   
Friedrichshain                   30         30         30             30   
Lichtenberg 

## Cluster Neighborhoods <a name="Cluster"></a>

In [27]:
#Let’s analyze each neighborhood to know about the top 5 venues of each one.
#one hot encoding
bl_onehot = pd.get_dummies(venues_df_restaurant[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bl_onehot['Neighborhoods'] = venues_df_restaurant['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bl_onehot.columns[-1]] + list(bl_onehot.columns[:-1])
bl_onehot = bl_onehot[fixed_columns]

print(bl_onehot.shape)
bl_onehot.head()

(1279, 59)


AttributeError: 'NoneType' object has no attribute 'items'

       Neighborhoods  African Restaurant  American Restaurant  \
1   Mitte (locality)                   0                    0   
11  Mitte (locality)                   0                    0   
14  Mitte (locality)                   0                    0   
17  Mitte (locality)                   0                    0   
21  Mitte (locality)                   0                    0   

    Argentinian Restaurant  Asian Restaurant  Austrian Restaurant  \
1                        0                 0                    0   
11                       0                 0                    0   
14                       0                 0                    0   
17                       0                 0                    0   
21                       0                 0                    1   

    Bavarian Restaurant  Brazilian Restaurant  Cajun / Creole Restaurant  \
1                     0                     0                          0   
11                    0                   

In [28]:
#Use pandas groupby on neighborhood column and calculate the mean of the frequency of occurrence of each venue category.
bl_grouped=bl_onehot.groupby("Neighborhoods").mean().reset_index()
bl_grouped

AttributeError: 'NoneType' object has no attribute 'items'

                Neighborhoods  African Restaurant  American Restaurant  \
0                   Adlershof            0.000000             0.000000   
1        Alt-Hohenschönhausen            0.000000             0.000000   
2                 Alt-Treptow            0.000000             0.000000   
3                Altglienicke            0.000000             0.000000   
4              Baumschulenweg            0.000000             0.000000   
5           Biesdorf (Berlin)            0.000000             0.000000   
6        Blankenburg (Berlin)            0.000000             0.000000   
7                   Bohnsdorf            0.000000             0.000000   
8                 Borsigwalde            0.000000             0.000000   
9                       Britz            0.000000             0.000000   
10              Buch (Berlin)            0.000000             0.000000   
11            Buckow (Berlin)            0.000000             0.000000   
12             Charlottenburg         

In [29]:
#Output each neighborhood along with the top 5 most common venues:
num_top_venues=5

for hood in bl_grouped['Neighborhoods']:
    print('---'+hood+'---')
    temp=bl_grouped[bl_grouped['Neighborhoods']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Adlershof---
               venue  freq
0  German Restaurant   0.3
1   Greek Restaurant   0.2
2   Sushi Restaurant   0.2
3  Korean Restaurant   0.1
4         Restaurant   0.1


---Alt-Hohenschönhausen---
                  venue  freq
0      Greek Restaurant  0.29
1     German Restaurant  0.29
2  Fast Food Restaurant  0.29
3     Indian Restaurant  0.14
4    African Restaurant  0.00


---Alt-Treptow---
                           venue  freq
0             Falafel Restaurant  0.22
1             Italian Restaurant  0.17
2      Middle Eastern Restaurant  0.11
3               Tapas Restaurant  0.11
4  Vegetarian / Vegan Restaurant  0.11


---Altglienicke---
                venue  freq
0    Greek Restaurant   0.4
1   Korean Restaurant   0.2
2    Sushi Restaurant   0.2
3          Restaurant   0.2
4  Russian Restaurant   0.0


---Baumschulenweg---
                   venue  freq
0     Falafel Restaurant  0.29
1  Vietnamese Restaurant  0.14
2     Italian Restaurant  0.14
3      Indian Restauran

                        venue  freq
0          Falafel Restaurant  0.07
1            Doner Restaurant  0.07
2  Modern European Restaurant  0.07
3          Italian Restaurant  0.07
4           Indian Restaurant  0.07


---Rosenthal (Berlin)---
                 venue  freq
0   Mexican Restaurant  0.25
1  American Restaurant  0.12
2     Asian Restaurant  0.12
3   Italian Restaurant  0.12
4     Greek Restaurant  0.12


---Rudow---
                venue  freq
0  Italian Restaurant  0.50
1    Greek Restaurant  0.33
2  Chinese Restaurant  0.17
3  African Restaurant  0.00
4  Russian Restaurant  0.00


---Rummelsburg---
                   venue  freq
0     Italian Restaurant  0.25
1  Vietnamese Restaurant  0.12
2       Asian Restaurant  0.12
3     Falafel Restaurant  0.08
4      Indian Restaurant  0.04


---Schmargendorf---
                   venue  freq
0     Italian Restaurant  0.55
1             Restaurant  0.10
2     Chinese Restaurant  0.10
3      German Restaurant  0.10
4  Vietnamese Rest

In [30]:
#Cluster Neighborhoods
#Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

# set number of clusters
kclusters = 5

bl_clustering = bl_grouped.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 3, 2, 3, 3, 2, 3, 3, 3], dtype=int32)

In [31]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bl_merged = bl_grouped.copy()

# add clustering labels
bl_merged["Cluster Labels"] = kmeans.labels_
#为何rename?
bl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bl_merged.head()

AttributeError: 'NoneType' object has no attribute 'items'

           Neighborhood  African Restaurant  American Restaurant  \
0             Adlershof                 0.0                  0.0   
1  Alt-Hohenschönhausen                 0.0                  0.0   
2           Alt-Treptow                 0.0                  0.0   
3          Altglienicke                 0.0                  0.0   
4        Baumschulenweg                 0.0                  0.0   

   Argentinian Restaurant  Asian Restaurant  Austrian Restaurant  \
0                     0.0               0.0                  0.0   
1                     0.0               0.0                  0.0   
2                     0.0               0.0                  0.0   
3                     0.0               0.0                  0.0   
4                     0.0               0.0                  0.0   

   Bavarian Restaurant  Brazilian Restaurant  Cajun / Creole Restaurant  \
0                  0.0                   0.0                        0.0   
1                  0.0          

In [32]:
bl_merged = bl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(bl_merged.shape)
bl_merged.head()

(95, 62)


AttributeError: 'NoneType' object has no attribute 'items'

           Neighborhood  African Restaurant  American Restaurant  \
0             Adlershof                 0.0                  0.0   
1  Alt-Hohenschönhausen                 0.0                  0.0   
2           Alt-Treptow                 0.0                  0.0   
3          Altglienicke                 0.0                  0.0   
4        Baumschulenweg                 0.0                  0.0   

   Argentinian Restaurant  Asian Restaurant  Austrian Restaurant  \
0                     0.0               0.0                  0.0   
1                     0.0               0.0                  0.0   
2                     0.0               0.0                  0.0   
3                     0.0               0.0                  0.0   
4                     0.0               0.0                  0.0   

   Bavarian Restaurant  Brazilian Restaurant  Cajun / Creole Restaurant  \
0                  0.0                   0.0                        0.0   
1                  0.0          

In [33]:
print(bl_merged.shape)
bl_merged.sort_values(["Cluster Labels"], inplace=True)
bl_merged

(95, 62)


AttributeError: 'NoneType' object has no attribute 'items'

                 Neighborhood  African Restaurant  American Restaurant  \
15        Falkenberg (Berlin)            0.000000             0.000000   
40         Kaulsdorf (Berlin)            0.000000             0.000000   
86        Wartenberg (Berlin)            0.000000             0.000000   
51           Malchow (Berlin)            0.000000             0.000000   
39             Karow (Berlin)            0.000000             0.000000   
42                Konradshöhe            0.000000             0.000000   
0                   Adlershof            0.000000             0.000000   
21            Friedrichshagen            0.000000             0.000000   
18       Französisch Buchholz            0.000000             0.000000   
37      Johannisthal (Berlin)            0.000000             0.000000   
16         Falkenhagener Feld            0.000000             0.000000   
59       Neu-Hohenschönhausen            0.000000             0.000000   
6        Blankenburg (Berlin)         

In [34]:
#Visualize the resulting clusters

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bl_merged['Latitude'], bl_merged['Longitude'], bl_merged['Neighborhood'], bl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
# save the map as HTML file
map_clusters.save('map_clusters_cap_final.html')

In [37]:
#Examine Clusters

In [38]:
#cluster 0
bl_merged.loc[bl_merged['Cluster Labels'] == 0]

AttributeError: 'NoneType' object has no attribute 'items'

           Neighborhood  African Restaurant  American Restaurant  \
15  Falkenberg (Berlin)                 0.0                  0.0   
40   Kaulsdorf (Berlin)                 0.0                  0.0   
86  Wartenberg (Berlin)                 0.0                  0.0   

    Argentinian Restaurant  Asian Restaurant  Austrian Restaurant  \
15                     0.0               0.0                  0.0   
40                     0.0               0.0                  0.0   
86                     0.0               0.0                  0.0   

    Bavarian Restaurant  Brazilian Restaurant  Cajun / Creole Restaurant  \
15                  0.0                   0.0                        0.0   
40                  0.0                   0.0                        0.0   
86                  0.0                   0.0                        0.0   

    Caribbean Restaurant  Caucasian Restaurant  Chinese Restaurant  \
15                   0.0                   0.0                 0.0   
40   

In [39]:
#Cluster 2
bl_merged.loc[bl_merged['Cluster Labels'] == 2]

AttributeError: 'NoneType' object has no attribute 'items'

             Neighborhood  African Restaurant  American Restaurant  \
0               Adlershof                 0.0                  0.0   
21        Friedrichshagen                 0.0                  0.0   
18   Französisch Buchholz                 0.0                  0.0   
37  Johannisthal (Berlin)                 0.0                  0.0   
16     Falkenhagener Feld                 0.0                  0.0   
59   Neu-Hohenschönhausen                 0.0                  0.0   
6    Blankenburg (Berlin)                 0.0                  0.0   
54                Marzahn                 0.0                  0.0   
1    Alt-Hohenschönhausen                 0.0                  0.0   
3            Altglienicke                 0.0                  0.0   

    Argentinian Restaurant  Asian Restaurant  Austrian Restaurant  \
0                      0.0          0.000000                  0.0   
21                     0.0          0.166667                  0.0   
18                    

In [40]:
#Cluster 3
bl_merged.loc[bl_merged['Cluster Labels'] == 3]

AttributeError: 'NoneType' object has no attribute 'items'

                Neighborhood  African Restaurant  American Restaurant  \
56                    Moabit            0.000000             0.000000   
65         Pankow (locality)            0.000000             0.000000   
64           Oberschöneweide            0.000000             0.000000   
63                Nikolassee            0.000000             0.000000   
62         Niederschönhausen            0.000000             0.000000   
61         Niederschöneweide            0.000000             0.000000   
57        Märkisches Viertel            0.000000             0.083333   
60       Neukölln (locality)            0.047619             0.000000   
53               Marienfelde            0.000000             0.000000   
66               Plänterwald            0.000000             0.000000   
55          Mitte (locality)            0.000000             0.000000   
52                Mariendorf            0.000000             0.058824   
67           Prenzlauer Berg            0.000000   

In [41]:
#Cluster 4
bl_merged.loc[bl_merged['Cluster Labels'] == 4]

AttributeError: 'NoneType' object has no attribute 'items'

                 Neighborhood  African Restaurant  American Restaurant  \
79  Stadtrandsiedlung Malchow                 0.0                  0.0   
32                 Haselhorst                 0.0                  0.0   
85                    Wannsee                 0.0                  0.0   
58                 Müggelheim                 0.0                  0.5   
44                   Köpenick                 0.0                  0.0   
74                Schmöckwitz                 0.0                  0.0   
24                      Gatow                 0.0                  0.0   
68                  Rahnsdorf                 0.0                  0.0   
33                Heiligensee                 0.0                  0.0   

    Argentinian Restaurant  Asian Restaurant  Austrian Restaurant  \
79                0.000000          0.000000             0.000000   
32                0.142857          0.142857             0.000000   
85                0.000000          0.000000        

## Conclusion <a name="conclusion"></a>

From the analysis we can establish that Vegetarian / Vegan Restaurant are not quite as popular by frequancy in Berlin as we might thought. The top five most popular types of resturants across the city were as follows:

1. Italian
2. German
3. Vietnamese
4. Greek
5. Falafel

Vegetarian / Vegan restaurants were 12th on the list of most frequant restaurants in the city. This might make you conclude that Berlin is less hippster than once thought. However this could be analysed in future research where Berlin is compared with other German cites of simlar size to see if it has comparably more or less Vegetarian / Vegan Restaurant. 

The cluster analysis also found that the neigbourhood with the highest frequancy of Vegetarian / Vegan restaurants was Neukölln, where it was actually the most frequent resturant type. This was also the case in Tiergarten. The neigbourhoods with the highest frequency are below:

1. Neukölln
2. Tiergarten
3. Friedrichshain
4. Lichtenberg
5. Fennpfuhl

Neukölln, Tiergarten and Friedrichshain are all inner city neigbourhoods with a higher leverl of young people but also hight levels of non-ethinc germans (https://en.wikipedia.org/wiki/Demographics_of_Berlin). We could conclude that if we look at the these neigbourhoods that thet are more young and hippster though a correlation of higher frequencey of Vegetarian / Vegan restaurants. However it is not conclusive as these neigbourhoods have higher levels of internationl residents which could be the reseaon for the higher frequency.  

